# T5.1 Panel de control con PowerBI
## Ralphy Núñez Mercado
Hai que facer un proxecto con PowerBI que teña os seguintes elementosPower BI:

Dous informes (e vista para móbiles e ordenador por cada informe).
Alomenos 2 gráficas ou elementos gráficos ordinarios de PowerBI.
1 Mapa (aínda que sexa con outros datos sen relación).
Dúas orixes de datos, por exemplo un excel e un JSON.
Relacións entre os datos (alomenos algunha que teña sentido).
Un obxecto visual de Python.
Un orixe de datos cun script de Python.
Emprega varios orixes de datos (diferentes) para comparar.

Na práctica debe verse algo de:

Scrapping
PANDAS
Spark-HDFS <-> PowerBi

E ademáis que quede bonito o deseño do informe :)

Deberase facer con PowerBI Desktop e publicarase a app.powerbi.com coa conta @fernandowirtz.com.
Subirase a esta tarefa o arquivo .pbix. Se é moi grande para subir, darase unha ligazón ao arquivo en OneDrive (coa conta @fernandowirtz.com).
Compartirase a ligazón ao informe publicado en app.powerbi.com tamén co profe (pode poñerse no texto da tarefa ou engadir ao membro da organización co email: scj@fernandowirtz.com)

### ⬇️ Instalar las librerías

In [ ]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true
!pip install pyodbc -y || true

^C


### 🦎 Instalar gecko Driver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\ralphy.nunezmercado\\.wdm\\drivers\\geckodriver\\win64\\v0.36.0\\geckodriver.exe'

### ⬇️ Importar las librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import itertools
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import json
from datetime import datetime
from sqlalchemy import create_engine, text
import pyodbc
import pandas as pd

### Conectar el driver de Firefox

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Iniciar navegador
options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)

# Ir á páxina
url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

# Espera a que cargue e aceptar cookies
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="as24-cmp-accept-all-button"]'))
    ).click()
    print("Cookies aceptadas.")
except Exception as e:
    print("Non se puideron aceptar as cookies:", e)

# Esperar á recarga (ou esperar polo elemento clave que indica que xa cargou)
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'cl-list-element-gap'))
    )
    print("Páxina cargada correctamente.")
except:
    print("Erro esperando polos resultados")

# Recolle os datos
coches = driver.find_elements(By.CLASS_NAME, 'cl-list-element-gap')
data = []

for coche in coches:
    try:
        titulo = coche.find_element(By.CSS_SELECTOR, 'h2[class*="Title"]').text
        precio = coche.find_element(By.CSS_SELECTOR, '[data-item-name="price"]').text
        detalles = coche.find_elements(By.CSS_SELECTOR, '[data-item-name="specs"] li')
        combustible = detalles[0].text if len(detalles) > 0 else "N/A"
        kilometros = detalles[1].text if len(detalles) > 1 else "N/A"
        ano = detalles[2].text if len(detalles) > 2 else "N/A"
        ubicacion = coche.find_element(By.CSS_SELECTOR, '[data-item-name="seller-location"]').text

        data.append({
            'Título': titulo,
            'Precio': precio,
            'Combustible': combustible,
            'Kilómetros': kilometros,
            'Año': ano,
            'Ubicación': ubicacion
        })
    except Exception as e:
        print("Erro procesando un coche:", e)
        continue

driver.quit()

# Gardar en CSV
df = pd.DataFrame(data)
df.to_csv("coches_autoscout24.csv", index=False)
print("Scraping completo. Gardados", len(df), "coches.")
print(df.head())


Error al navegar entre páginas: Message: Unable to locate element: .next; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:527:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

